In [149]:
from collections import defaultdict
import sys
import random
import numpy as np
import os #for data load
from scienceie_loader import load_tokenized_data, load_data_with_char_offsets #for data preparing also customize dataset requirments for load

In [150]:
#dataset loading from data folder
data_root = 'data'
data_train = os.path.join(data_root, 'train2')
data_test = os.path.join(data_root, 'semeval_articles_test')

In [151]:
#spliting into train and test dataset and with customize function that comes with dataset
# and this function define in scienceie_loader.py
train_docs, train_rels, _ = load_tokenized_data(data_train)
test_docs, test_rels, _ = load_tokenized_data(data_test)

# Prepareing Data for HMM
1. Data preparing

In [152]:
wordtag = defaultdict(int) # emission freqs
unitag = defaultdict(int) # unigram freqs of tags
bitag = defaultdict(int) # bigram freqs of tags
tritag = defaultdict(int) # trigram freqs of tags

tag_penult = ''
tag_last = ''
tag_current = ''

for l in train_docs:
    for word,tag in l:
        if not l:
            tag_penult = tag_last
            tag_last = tag_current
            tag_current = ''
            # update sentence boundary case
            if tag_last != '' and tag_penult != '':
                # update bitag freqs
                bitag[(tag_last, tag_current)] += 1
                # update tritag freqs
                tritag[(tag_penult, tag_last, tag_current)] += 1
        else:
            tag_penult = tag_last
            tag_last = tag_current
            tag_current = tag
            # update emission freqs
            wordtag[(word,tag)] += 1
            # update unitag freqs
            unitag[tag] += 1
            # update bitag freqs
            bitag[(tag_last, tag_current)] += 1
            # update tritag freqs
            tritag[(tag_penult, tag_last, tag_current)] += 1
            # update starting bigrams
            if tag_last == '' and tag_penult == '':
                bitag[('','')] += 1

In [153]:
word,tag=train_docs[0][0][0], train_docs[0][0][1]
print(float(wordtag[(word,tag)])/unitag[tag])
print(float(tritag[(tag_penult, tag_last, tag_current)])/bitag[(tag_penult, tag_last)])

2.2850875188519722e-05
0.871086312671885


2. Define method and train

In [154]:
#unknown word handeling method start
if 'UNK':
    new = defaultdict(int)
    # change words with freq <5 into unknown words "<UNK>"
    for (word,tag) in wordtag:
        new[(word,tag)] = wordtag[(word,tag)]
        if wordtag[(word,tag)] < 5:
            new[('<UNK>',tag)] += wordtag[(word,tag)]
    wordtag = new
#end

#train data
words = set([key[0] for key in wordtag.keys()])
tags = set(unitag.keys())
E = defaultdict(int)
Q = defaultdict(int)
for (word,tag) in wordtag:
    E[(word,tag)] = float(wordtag[(word,tag)])/unitag[tag]
for (tag_penult, tag_last, tag_current) in tritag:
    Q[(tag_penult, tag_last, tag_current)] = float(tritag[(tag_penult, tag_last, tag_current)])/bitag[(tag_penult, tag_last)]

3. Viterbi Algorithm for tagging

In [155]:
    def viterbi(sent,method='UNK'):
        V = {}
        path = {}
        # init
        V[0,'',''] = 1
        path['',''] = []
        # run
        for k in range(1,len(sent)+1):
            temp_path = {}
            word = get_word(sent,k-1)
            # handling unknown words in test set using low freq words in training set
            if word not in words:
                if method=='UNK':
                    word = '<UNK>'
            for u in get_possible_tags(k-1):
                for v in get_possible_tags(k):
                    V[k,u,v],prev_w = max([(V[k-1,w,u] * Q[w,u,v] * E[word,v],w) for w in get_possible_tags(k-2)])
                    temp_path[u,v] = path[prev_w,u] + [v]
            path = temp_path
        # last step
        prob,umax,vmax = max([(V[len(sent),u,v] * Q[u,v,''],u,v) for u in tags for v in tags])
        return path[umax,vmax]

In [156]:
    def get_possible_tags(k):
        if k == -1:
            return set([''])
        if k == 0:
            return set([''])
        else:
            return tags

    def get_word(sent,k):
        if k < 0:
            return ''
        else:
            return sent[k]

In [157]:
#Appending test docs in an array for predict
sent=[]
s_tags=[]
for i in test_docs:
    s=[]
    st=[]
    for word,tag in i:
        s.append(word)
        st.append(tag)
    sent.append(s)
    s_tags.append(st)

In [166]:
#Evaluating
CP=NP=0
for i in range(len(sent)):
    path=viterbi(sent[i], "UNK")
    for j in range(len(sent[i])):
        if s_tags[i][j]==path[j]:
            CP+=1
        NP+=1
acc=CP/NP
print(acc*100)

75.82120680665497


In [165]:
#predicting
i=random.randint(0,len(sent))
path=viterbi(sent[0], method='UNK')
for word,p_tags in zip(sent[0],path):
    print(word, p_tags)

First O
results O
from O
RHIC B-Task
on I-Task
charged I-Task
multiplicities I-Task
, I-Task
evolution I-Task
of I-Task
multiplicities I-Task
with I-Task
centrality I-Task
, O
particle O
ratios O
and O
transverse B-Process
momentum I-Process
distributions I-Process
in O
central O
and O
minimum O
bias O
collisions B-Process
, O
are O
analyzed O
in O
a O
string B-Process
model I-Process
which O
includes O
hard B-Process
collisions I-Process
, O
collectivity O
in O
the O
initial O
state O
considered O
as O
string B-Process
fusion I-Process
, O
and O
rescattering B-Task
of I-Task
the I-Task
produced I-Task
secondaries. I-Task
Multiplicities I-Task
and I-Task
their I-Task
evolution I-Task
with I-Task
centrality I-Task
are O
successfully O
reproduced. B-Process
Transverse I-Process
momentum I-Process
distributions I-Process
in I-Process
the I-Process
model I-Process
show O
a O
larger O
pT-tail O
than O
experimental B-Material
data I-Material
, O
disagreement O
which O
grows O
with O
increasi